In [1]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0005.hk',"2017-09-28","2021-09-24")
# data.drop("Adj Close",axis=1,inplace=True)
data.tail()

[*********************100%***********************]  1 of 1 completed


Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2021-09-16  40.599998  40.599998  39.599998  40.000000  40.000000  19553563
2021-09-17  39.700001  40.400002  39.099998  40.250000  40.250000  28266370
2021-09-20  39.900002  39.950001  38.950001  39.000000  39.000000  23796781
2021-09-21  38.349998  39.000000  38.299999  38.799999  38.799999  18117762
2021-09-23  39.299999  39.950001  39.200001  39.750000  39.750000  20518033

In [3]:
data['prediction'] = data['Close'].shift(-1)
data.dropna(inplace=True)
data

Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2017-09-28  76.400002  77.099998  76.050003  76.199997  64.387009  22520013   
2017-09-29  76.500000  76.900002  76.300003  76.550003  64.682747  16766252   
2017-10-03  76.800003  77.449997  76.699997  77.449997  65.443207  24369212   
2017-10-04  77.500000  77.949997  77.500000  77.849998  65.781189  21197563   
2017-10-06  77.599998  77.800003  77.199997  77.349998  65.358711  23715110   
...               ...        ...        ...        ...        ...       ...   
2021-09-15  40.599998  40.599998  40.000000  40.099998  40.099998  14418368   
2021-09-16  40.599998  40.599998  39.599998  40.000000  40.000000  19553563   
2021-09-17  39.700001  40.400002  39.099998  40.250000  40.250000  28266370   
2021-09-20  39.900002  39.950001  38.950001  39.000000  39.000000  23796781   
2021-09-21  38.349998  39.000000  38.299999  38.799999  38.799999  18117762   

            prediction  
Date                    
2017-09-28   76.550003  
2017-09-29   77.449997  
2017-10-03   77.849998  
2017-10-04   77.349998  
2017-10-06   77.500000  
...                ...  
2021-09-15   40.000000  
2021-09-16   40.250000  
2021-09-17   39.000000  
2021-09-20   38.799999  
2021-09-21   39.750000  

[982 rows x 7 columns]

In [4]:
#Predicting the Stock price in the future
X = data.drop(['prediction'], 1)
Y = np.array(data['prediction'])
X_prediction = X[-5:]
X_prediction

Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2021-09-15  40.599998  40.599998  40.000000  40.099998  40.099998  14418368
2021-09-16  40.599998  40.599998  39.599998  40.000000  40.000000  19553563
2021-09-17  39.700001  40.400002  39.099998  40.250000  40.250000  28266370
2021-09-20  39.900002  39.950001  38.950001  39.000000  39.000000  23796781
2021-09-21  38.349998  39.000000  38.299999  38.799999  38.799999  18117762

In [7]:
 X_train,X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

#Performing the Regression on the training data
clf = LinearRegression()
clf.fit(X_train, Y_train)
prediction = (clf.predict(X_prediction))

last_row = data.tail(1)
print(last_row['Close'])

Date
2021-09-21    38.799999
Name: Close, dtype: float64


In [8]:
prediction

array([40.1712522 , 39.9283844 , 40.02324547, 38.98440022, 38.83603789])

In [10]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
df = yf.download('0005.hk',"2021-09-22","2021-09-30")
# data.drop("Adj Close",axis=1,inplace=True)
df.tail()

[*********************100%***********************]  1 of 1 completed


Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2021-09-23  39.299999  39.950001  39.200001  39.750000  39.750000  20518033
2021-09-24  40.200001  40.799999  39.700001  39.700001  39.700001  19052618
2021-09-27  40.000000  41.099998  40.000000  40.299999  40.299999  20332913
2021-09-28  41.250000  41.250000  40.599998  40.849998  40.849998  29545281
2021-09-29  40.500000  41.599998  40.299999  41.250000  41.250000  30287737